In [13]:
{ tuple(np.load('./database/face_encodings/'+encoded_face_id)):encoded_face_id.split('.')[0] for encoded_face_id in os.listdir('./database/face_encodings/') }

{(-0.13267342746257782,
  0.16745693981647491,
  0.035298481583595276,
  -0.033928144723176956,
  -0.1301245540380478,
  -0.07134506851434708,
  -0.14142200350761414,
  -0.18150505423545837,
  0.08887521922588348,
  -0.09880905598402023,
  0.25074806809425354,
  -0.029477424919605255,
  -0.2514168620109558,
  -0.11865878105163574,
  -0.05802131071686745,
  0.1247931569814682,
  -0.20628738403320312,
  -0.13798443973064423,
  0.0005696872249245644,
  -0.08893042802810669,
  0.13348783552646637,
  0.03435872122645378,
  0.09697136282920837,
  0.011655408889055252,
  -0.09127835929393768,
  -0.4120059311389923,
  -0.09867683798074722,
  -0.12842053174972534,
  0.002485324628651142,
  -0.07892625778913498,
  0.03152521327137947,
  0.008021438494324684,
  -0.18162468075752258,
  -0.019821377471089363,
  -0.011953884735703468,
  0.02544783614575863,
  0.017939290031790733,
  0.0017304476350545883,
  0.14911764860153198,
  -0.0006059017032384872,
  -0.19194328784942627,
  0.01694880798459053,

In [8]:
import face_recognition
import cv2
import numpy as np
import pandas as pd
import ast
import sys
import os

class Attendance:

    def __init__(self, scale_frame=0.5, face_location_model='hog', face_encoding_model = 'small'):

        def retrieve_faces_encodings():
            '''Retrieves and retuns dictionary (key is face enoding and value is the student id) of faces encoding from the server'''
            return { np.load('./database/face_encodings/'+encoded_face_id):encoded_face_id.split('-')[0] for encoded_face_id in os.listdir('./database/face_encodings/') }

        self.__encodings_database = retrieve_faces_encodings()

        self.__identified_student_ids = []
        self.scale_frame = scale_frame

        self.process_current_frame = True
        self.face_location_model = face_location_model #'cnn' has better accuracy but uses GPU, 'hog' is faster with less accuracy uses cpu
        self.face_encoding_model = face_encoding_model #'large' model has better accuracy but is slower, 'small' model is faster

    # def read_image_path(img_path: str) -> np.array:
    #     '''Reads given image path and returns its numpy array representation'''
    #     return cv2.imread(img_path)

    def image_array_to_face_encoding(image_numpy_arr: np.array) -> np.array:
        '''Inputs numpy array representation of an image and returns the 122 dimentional (numpy) encoding of every faces in the picture in a list'''
        return face_recognition.face_encodings(image_numpy_arr)
    
    def start_session(self, camera_index=0):
        self.__encodings_database_encodings_only = [self.__encodings_database.keys()] # Getting faces encodings only from the database
        cap = cv2.VideoCapture(camera_index)
        while True:
            ret, frame = cap.read()

            small_frame = cv2.resize(frame, (0, 0), fx=self.scale_frame, fy=self.scale_frame) # Resize the frame for faster processing
            rgb_frame = small_frame[:, :, ::-1] # Convert the frame from BGR to RGB

            face_locations = face_recognition.face_locations(rgb_frame, model=self.face_location_model) # Find face locations and face encodings in the frame
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations) # Generate encodings of every faces in the frame in a list
            
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(self.__encodings_database_encodings_only, face_encoding) # Compare the face encoding with the list of known encoded faces
                if True in matches:
                    match_index = matches.index(True)
                    identity = self.__encodings_database_encodings_only[match_index]
                else:
                    identity = 'Unknown'

                self.__identified_student_ids.append(identity)

            print(self.__identified_student_ids)
            self.__identified_student_ids = []


            
            # Draw rectangles around detected faces and display names
            for (top, right, bottom, left) in face_locations:
                top *= int(1 / self.scale_frame)
                right *= int(1 / self.scale_frame)
                bottom *= int(1 / self.scale_frame)
                left *= int(1 / self.scale_frame)
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # Display the frame with face rectangles
            cv2.imshow('Face Detection', frame)
            
            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the camera and close the window
        cap.release()
        cv2.destroyAllWindows()
Session = Attendance()
Session.start_session()

TypeError: unhashable type: 'numpy.ndarray'

In [12]:
# import os
# import numpy as np

# face_encodings_path = './database/face_encodings/'
# known_faces = [ np.load(face_encodings_path+encoded_face) for encoded_face in os.listdir(face_encodings_path) ]
# known_faces[0].shape

tuple(np.array([1,2]))

(1, 2)

In [5]:
import cv2
import matplotlib.pyplot as plt
import face_recognition
import numpy as np

def read_image_path(img_path: str) -> np.array:
    return cv2.imread(img_path)

def image_array_to_face_encoding(image_numpy_arr: np.array) -> np.array:
    return face_recognition.face_encodings(image_numpy_arr)

def run_recognization(scale_frame=0.5, preview=True):

    if preview==True:
        # Open the default camera (usually the built-in webcam)
        cap = cv2.VideoCapture(0)

        target = []

        while True:
            # Capture a frame from the camera
            ret, frame = cap.read()
            
            # Resize the frame for faster processing
            small_frame = cv2.resize(frame, (0, 0), fx=scale_frame, fy=scale_frame)
            
            # Convert the frame from BGR to RGB
            rgb_frame = small_frame[:, :, ::-1]
            
            # Find face locations and face encodings in the frame
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            # print(len(face_encodings), face_encodings)




            # Initialize the match status as "Unknown"
            match_status = None
            
            # Match detected faces against known faces
            for face_encoding in face_encodings:
                # Compare the face encoding with the list of known encoded faces
                matches = face_recognition.compare_faces(known_faces, face_encoding)
                # print(len(matches), matches)
                
                if True in matches:
                    # match_indexes = [idx for idx, value in enumerate(matches) if value is True]

                    match_status = matches.index(True)
                    # print(match_indexes)
                else:
                    match_status = 'Unknown'

                target.append(match_status)

            print(target)
            target = []



            
            # Draw rectangles around detected faces and display names
            for (top, right, bottom, left) in face_locations:
                top *= int(1 / scale_frame)
                right *= int(1 / scale_frame)
                bottom *= int(1 / scale_frame)
                left *= int(1 / scale_frame)
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # Display the frame with face rectangles
            cv2.imshow('Face Detection', frame)
            
            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the camera and close the window
        cap.release()
        cv2.destroyAllWindows()
    else:
        False

run_recognization()




[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [ ]:
+